<a href="https://colab.research.google.com/github/amit154154/Amit-Israeli/blob/master/exrecies4_Intro_data_engenier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import modules



In [1]:
import json
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns 


# Set kaggle api and download the datasets



In [2]:
!pip install kaggle
!mkdir /root/.kaggle/ 


username = 'amitisraeli'
api_key = '2f3ca193658687f4b69ec3ab61ac49d4'

api_token = {"username":username, "key":api_key}

with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

**download datasets:**


*   The Marvel Universe Social Network
*   Significant Earthquakes 1965-2016
*   FIFA 19 complete player dataset




In [55]:
!mkdir ./datasets
!mkdir ./datasets/Marvel_Universe_Social_Network
!mkdir ./datasets/Significant_Earthquakes
!mkdir ./datasets/Fifa_database

!kaggle datasets download csanhueza/the-marvel-universe-social-network  -p ./datasets/Marvel_Universe_Social_Network
!kaggle datasets download usgs/earthquake-database  -p ./datasets/Significant_Earthquakes
!kaggle datasets download karangadiya/fifa19  -p ./datasets/Fifa_database

!unzip ./datasets/Marvel_Universe_Social_Network/*.zip -d ./datasets/Marvel_Universe_Social_Network/database/
!unzip ./datasets/Significant_Earthquakes/*.zip -d ./datasets/Significant_Earthquakes/database/
!unzip ./datasets/Fifa_database/*.zip -d ./datasets/Fifa_database/database/

mkdir: cannot create directory ‘./datasets’: File exists
mkdir: cannot create directory ‘./datasets/Marvel_Universe_Social_Network’: File exists
mkdir: cannot create directory ‘./datasets/Significant_Earthquakes’: File exists
the-marvel-universe-social-network.zip: Skipping, found more recently modified local copy (use --force to force download)
earthquake-database.zip: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/2.18M [00:00<?, ?B/s]
100% 2.18M/2.18M [00:00<00:00, 72.7MB/s]
Archive:  ./datasets/Marvel_Universe_Social_Network/the-marvel-universe-social-network.zip
replace ./datasets/Marvel_Universe_Social_Network/database/edges.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./datasets/Marvel_Universe_Social_Network/database/edges.csv  
replace ./datasets/Marvel_Universe_Social_Network/database/hero-network.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./datasets/Marvel_Universe_Social_Network/database/hero-network.c

# Marvel_Universe_Social_Network

In [104]:
df_nodes = pd.read_csv('/content/datasets/Marvel_Universe_Social_Network/database/edges.csv')
df_nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96104 entries, 0 to 96103
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hero    96104 non-null  object
 1   comic   96104 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


# Significant_Earthquakes dataset


setting dataframe

In [25]:
df = pd.read_csv('/content/datasets/Significant_Earthquakes/database/database.csv')
df

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,Magnitude Error,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.2460,145.6160,Earthquake,131.60,NaN,NaN,6.0,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.8630,127.3520,Earthquake,80.00,NaN,NaN,5.8,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.5790,-173.9720,Earthquake,20.00,NaN,NaN,6.2,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.0760,-23.5570,Earthquake,15.00,NaN,NaN,5.8,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.9380,126.4270,Earthquake,15.00,NaN,NaN,5.8,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,12/28/2016,08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40.0,5.6,ML,0.320,18.0,42.47,0.120,NaN,0.1898,NN00570710,NN,NN,NN,Reviewed
23408,12/28/2016,09:13:47,38.3777,-118.8957,Earthquake,8.80,2.0,33.0,5.5,ML,0.260,18.0,48.58,0.129,NaN,0.2187,NN00570744,NN,NN,NN,Reviewed
23409,12/28/2016,12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,NaN,5.9,MWW,NaN,NaN,91.00,0.992,4.8,1.5200,US10007NAF,US,US,US,Reviewed
23410,12/29/2016,22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,NaN,6.3,MWW,NaN,NaN,26.00,3.553,6.0,1.4300,US10007NL0,US,US,US,Reviewed


get strongest Earthquake

In [5]:
df[df['Type'] == 'Earthquake'].sort_values('Magnitude',ascending=False).iloc[0]


Date                                            03/11/2011
Time                                              05:46:24
Latitude                                            38.297
Longitude                                          142.373
Type                                            Earthquake
Depth                                                   29
Depth Error                                            NaN
Depth Seismic Stations                                 541
Magnitude                                              9.1
Magnitude Type                                         MWW
Magnitude Error                                        NaN
Magnitude Seismic Stations                             NaN
Azimuthal Gap                                          9.5
Horizontal Distance                                    NaN
Horizontal Error                                       NaN
Root Mean Square                                      1.16
ID                            OFFICIAL20110311054624120_

strongest necluer Earthquake

In [6]:
df[df['Type'] == 'Nuclear Explosion'].sort_values('Magnitude',ascending=False).iloc[0]

Date                                 10/27/1973
Time                                   06:59:57
Latitude                                 70.779
Longitude                                54.177
Type                          Nuclear Explosion
Depth                                         0
Depth Error                                 NaN
Depth Seismic Stations                      NaN
Magnitude                                   6.9
Magnitude Type                               MB
Magnitude Error                             NaN
Magnitude Seismic Stations                  NaN
Azimuthal Gap                               NaN
Horizontal Distance                         NaN
Horizontal Error                            NaN
Root Mean Square                            NaN
ID                                   USP00003YA
Source                                       US
Location Source                              US
Magnitude Source                             US
Status                                 R

set new column - 'Year' and setting pandas groupby object

In [8]:
df['Year'] = df['Date'].str.split('/').str[2]
year_group = df.groupby(['Year'])

print the year that had the least earthqueks that have more than 6 Magnitude


In [9]:
years = pd.Series(df['Year'].unique()).dropna()
years_count = []
for year in years:
  year_df = year_group.get_group(str(year))
  years_count.append(len(year_df[year_df['Magnitude'] > 6]))
print(years[years_count.index(min(years_count))])

1976


print the year that had the most earthqueks that have more than 9 Magnitude


In [10]:
years_count = []
for year in years:
  year_df = year_group.get_group(str(year))
  years_count.append(len(year_df[year_df['Magnitude'] > 9]))
print(years[years_count.index(max(years_count))])

2003


creat new column - 'Hour'

In [52]:
df['Hour'] = df['Time'].str.split(':').str[0]
df = df[df.Hour.str.len() < 3]
Hour_group = df.groupby(['Hour'])


print year with the most earthqueks

In [53]:
hours = pd.Series(df['Hour'].unique()).dropna()
hours_count = []
for hour in hours:
  hours_count.append(len(Hour_group.get_group(hour)))
print(hours[hours_count.index(max(hours_count))])

13


# FIFA 19 complete player dataset


setting dataframe


In [93]:
df = pd.read_csv('/content/datasets/Fifa_database/database/data.csv')
df

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,...,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,...,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,...,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,...,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,18202,238813,J. Lundstram,19,https://cdn.sofifa.org/players/4/19/238813.png,England,https://cdn.sofifa.org/flags/14.png,47,65,Crewe Alexandra,https://cdn.sofifa.org/teams/2/light/121.png,€60K,€1K,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,

function that return the strongest player in country

*  **input** - country name in dataframe
*  **output** - strongest player in that country



In [96]:
def strongest_in_country(country):
  df = pd.read_csv('/content/datasets/Fifa_database/database/data.csv')
  df = df[df['Nationality'] == country]
  return df.loc[df['Strength'].idxmax()]

In [97]:
print(strongest_in_country('Brazil'))
print('\n')
print(strongest_in_country('France'))

Unnamed: 0                                                  1124
ID                                                        232381
Name                                                      Wesley
Age                                                           21
Photo             https://cdn.sofifa.org/players/4/19/232381.png
                                       ...                      
GKHandling                                                    13
GKKicking                                                     14
GKPositioning                                                  8
GKReflexes                                                     6
Release Clause                                              €23M
Name: 1124, Length: 89, dtype: object


Unnamed: 0                                                  1193
ID                                                        202750
Name                                                     W. Boly
Age                                               

function that return avarge overall of all players in club

*  **input** - club name in dataframe
*  **output** - avarge overall of all player in that country


In [102]:
def Club_overall(club_name):
  df = pd.read_csv('/content/datasets/Fifa_database/database/data.csv')
  df = df[df['Club'] == club_name]
  return df['Overall'].sum() / len(df)
